In [0]:
#pivot is like any other simple pivot in excel,sql etc.

from pyspark.sql.functions import when,count,min,max,sum
data =[(1,'ram','M',1000,'IT'),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',3000,'IT'),(4,'Mohani','F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df=spark.createDataFrame(data,sch)
display(df)
df.groupBy(['dept_name','gender']).count().show()
df.groupBy('dept_name').pivot('gender').count().show() # pivot on all values of gender column
df.groupBy('dept_name').pivot('gender',['M']).count().show() # pivot on a specific value for example 'M' here

id,name,gender,salary,Dept_name
1,ram,M,1000,IT
2,Shyam,M,2000,HR
3,Mohana,F,3000,IT
4,Mohani,F,5000,HR


+---------+------+-----+
|dept_name|gender|count|
+---------+------+-----+
|       IT|     M|    1|
|       HR|     M|    1|
|       IT|     F|    1|
|       HR|     F|    1|
+---------+------+-----+

+---------+---+---+
|dept_name|  F|  M|
+---------+---+---+
|       HR|  1|  1|
|       IT|  1|  1|
+---------+---+---+

+---------+---+
|dept_name|  M|
+---------+---+
|       HR|  1|
|       IT|  1|
+---------+---+



In [0]:
#32: Unpivot Dataframe
# rotating columns into rows, there is no direct unpivot function , which can be achieved using stack()
sch=['Dept','no_males','no_females']
data =[('IT','8','9'),('HR','10','20'),('Payrol','5','1')]
df=spark.createDataFrame(data,sch)
display(df)
from pyspark.sql.functions import expr
unpivot_df=df.select('Dept',expr("stack(2,'Male',no_males,'Female',no_females) as (gender,count)"))
unpivot_df.show()

Dept,no_males,no_females
IT,8,9
HR,10,20
Payrol,5,1


+------+------+-----+
|  Dept|gender|count|
+------+------+-----+
|    IT|  Male|    8|
|    IT|Female|    9|
|    HR|  Male|   10|
|    HR|Female|   20|
|Payrol|  Male|    5|
|Payrol|Female|    1|
+------+------+-----+



In [0]:
# 33. Fill and fillna
# when you want to replace nulls/none values inside a dataframe
data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df=spark.createDataFrame(data,sch)
display(df)
# df.printSchema()
df_strng=df.fillna("No_Value").show()
display(df_strng)
# final_df=df_strng.fillna(0, subset=["salary"])
# display(final_df)
# df_filled = df.fillna({"salary": "No_Value"}).na.replace("salary", {None: "No_Value"})
df_filled = df_strng.fillna(0, subset=["salary"])
display(df_filled)
# df.na.fill("No_Value_Given").show()

id,name,gender,salary,Dept_name
1,ram,M,1000,null
2,Shyam,M,2000,HR
3,Mohana,F,null,IT
4,null,F,5000,HR


+---+--------+------+------+---------+
| id|    name|gender|salary|Dept_name|
+---+--------+------+------+---------+
|  1|     ram|     M|  1000| No_Value|
|  2|   Shyam|     M|  2000|       HR|
|  3|  Mohana|     F|  null|       IT|
|  4|No_Value|     F|  5000|       HR|
+---+--------+------+------+---------+

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-3217848536609604>", line 13, in <module>
    df_filled = df_strng.fillna(0, subset=["salary"])
AttributeError: 'NoneType' object has no attribute 'fillna'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1006, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/databricks/python/lib/p

In [0]:
# from pyspark.sql.functions import col

# # Replace both None and null values in the "salary" column with "No_Value"
# df_filled = df.fillna({"salary": "No_Value"}).na.replace("salary", {None: "No_Value"})

# df_filled.show()

data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df=spark.createDataFrame(data,sch)

# Replace both None and null values in the "salary" column with "No_Value"
df_filled = df.fillna(0, subset=["salary"])

# Show the DataFrame
df_filled.show()

+---+------+------+------+---------+
| id|  name|gender|salary|Dept_name|
+---+------+------+------+---------+
|  1|   ram|     M|  1000|     null|
|  2| Shyam|     M|  2000|       HR|
|  3|Mohana|     F|     0|       IT|
|  4|  null|     F|  5000|       HR|
+---+------+------+------+---------+



In [0]:
data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df = spark.createDataFrame(data, sch)

# Define default values for each data type
default_values = {
    "name": "NO_VALUE",
    "gender": "NO_VALUE",
    "salary": 0,
    "Dept_name": "NO_VALUE"
}

# Replace None values with default values
for col_name, default_val in default_values.items():
    df = df.fillna(default_val, subset=[col_name])

# Show the DataFrame
df.show()


+---+--------+------+------+---------+
| id|    name|gender|salary|Dept_name|
+---+--------+------+------+---------+
|  1|     ram|     M|  1000| NO_VALUE|
|  2|   Shyam|     M|  2000|       HR|
|  3|  Mohana|     F|     0|       IT|
|  4|NO_VALUE|     F|  5000|       HR|
+---+--------+------+------+---------+



In [0]:
data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df = spark.createDataFrame(data, sch)

# Replace None values with default values for each column
df_filled = df.fillna("NO_VALUE", subset=["name", "gender", "Dept_name"]) \
              .fillna(0, subset=["salary"])

# Show the DataFrame
df_filled.show()


+---+--------+------+------+---------+
| id|    name|gender|salary|Dept_name|
+---+--------+------+------+---------+
|  1|     ram|     M|  1000| NO_VALUE|
|  2|   Shyam|     M|  2000|       HR|
|  3|  Mohana|     F|     0|       IT|
|  4|NO_VALUE|     F|  5000|       HR|
+---+--------+------+------+---------+



In [0]:
#34. sample :sampling data from large dataset using fractio and seed

df=spark.range(1,101)
display(df)
df.sample(fraction=0.1).show()
# df.sample(fraction=0.1,seed=10).show() # shows same random value each time you execute

# fraction=0.1: This parameter specifies the fraction of rows to be sampled. Here, 0.1 means 10%, indicating that we want to sample 10% of the DataFrame's rows.
# seed=10: This parameter sets the seed for reproducibility. When a seed is specified, the sampling process will generate the same random sample each time it's executed with the same seed value. This is useful for reproducibility purposes. In this case, seed=10 sets the seed value to 10.

id
1
2
3
4
5
6
7
8
9
10


+---+
| id|
+---+
|  7|
|  8|
| 18|
| 54|
| 55|
| 56|
| 78|
| 80|
| 82|
| 87|
| 89|
| 91|
| 97|
+---+



In [0]:
#35. Collect : these are actions not transformation ..it returns an array/list of rows
#use collect for small dataset not for big datasets
data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df = spark.createDataFrame(data, sch)
display(df)
list_row=df.collect()
print(type(list_row))
print(list_row[0][1])


id,name,gender,salary,Dept_name
1,ram,M,1000,null
2,Shyam,M,2000,HR
3,Mohana,F,null,IT
4,null,F,5000,HR


<class 'list'>
ram


In [0]:
#36.Dataframe.transform functions :
# it is used to chain the custom transformations/user defined functions and this function returns the new dataframe after applying the specified transformations
#37: pyspark.sql.functions.transform() : this can be applied on a collumn which is an array type. it will perform transformation and then give you back a column with array type column

#38. CreateOrReplace TempView() : you can use sql queries like DB 
# Create a temporary view, these views are session scoped and cannot be shared between the sessions.

data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df = spark.createDataFrame(data, sch)
display(df)

df.createOrReplaceTempView("employees");
sql_df=spark.sql("""
                 SELECT *,
                 case
                 when e.gender ='M' then 'Male'
                 else 'Female'
                 end as Gender_Category
                 FROM employees e where salary is null
                 """
                 )
display(sql_df)

# once the temporary view is created you can utilize normal sql with magic command as below:


id,name,gender,salary,Dept_name
1,ram,M,1000,null
2,Shyam,M,2000,HR
3,Mohana,F,null,IT
4,null,F,5000,HR


id,name,gender,salary,Dept_name,Gender_Category
3,Mohana,F,null,IT,Female


In [0]:
%sql
                 SELECT *,
                 case
                 when e.gender ='M' then 'Male'
                 else 'Female'
                 end as Gender_Category
                 FROM employees e  

id,name,gender,salary,Dept_name,Gender_Category
1,ram,M,1000,null,Male
2,Shyam,M,2000,HR,Male
3,Mohana,F,null,IT,Female
4,null,F,5000,HR,Female


In [0]:
#39. CreateOrReplaceGlobalTempView():
# it can be access across the session with in spark application
# to acces these we need append global_temp.<tablename>

data =[(1,'ram','M',1000,None),(2,'Shyam','M',2000,'HR'),(3,'Mohana','F',None,'IT'),(4,None,'F',5000,'HR')]
sch=['id','name','gender','salary','Dept_name']
df = spark.createDataFrame(data, sch)
display(df)

df.createOrReplaceGlobalTempView("global_employees");
sql_df=spark.sql("""
                 SELECT *,
                 case
                 when e.gender ='M' then 'Male'
                 else 'Female'
                 end as Gender_Category
                 FROM employees e where salary >2000
                 """
                 )
display(sql_df)

id,name,gender,salary,Dept_name
1,ram,M,1000,null
2,Shyam,M,2000,HR
3,Mohana,F,null,IT
4,null,F,5000,HR


id,name,gender,salary,Dept_name,Gender_Category
4,null,F,5000,HR,Female


In [0]:
%sql
--SELECT * from global_employees --The table or view `global_employees` cannot be found
SELECT * from global_temp.global_employees; --this will work so to acces global temp table we need append 'global_temp' in front of name
-- this table will be acces in another notebook session too


id,name,gender,salary,Dept_name
1,ram,M,1000,null
2,Shyam,M,2000,HR
3,Mohana,F,null,IT
4,null,F,5000,HR


In [0]:
spark.catalog.currentDatabase()

Out[76]: 'default'

In [0]:
spark.catalog.listTables()

Out[77]: [Table(name='employees', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
spark.catalog.listTables('global_temp')

Out[78]: [Table(name='employees', catalog=None, namespace=['global_temp'], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='global_employees', catalog=None, namespace=['global_temp'], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='employees', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]